In [1]:
%run prepareStats.py

Last GP in the database: 2022 Canadian Grand Prix


In [2]:
dateOfDebut = pd.Series(results.groupby(["driverId"])["date"].min(), name="dateOfDebut")
dateOfFarewell = pd.Series(results.groupby(["driverId"])["date"].max(), name="dateOfFarewell")
dateOfFirstWin = pd.Series(results[results["position"] == 1].groupby(["driverId"])["date"].min(), name="dateOfFirstWin")
dateOfLastWin = pd.Series(results[results["position"] == 1].groupby(["driverId"])["date"].max(), name="dateOfLastWin")
results = results.merge(dateOfDebut, on = ["driverId"], how = "right")
results = results.merge(dateOfFarewell, on = ["driverId"], how = "right")
results = results.merge(dateOfFirstWin, on = ["driverId"], how = "right")
results = results.merge(dateOfLastWin, on = ["driverId"], how = "right")
results["careerLenght"] = results["dateOfFarewell"] - results["dateOfDebut"]

In [3]:
decidingraces = pd.read_csv(os.path.join("dataHandcollected", "titles.csv"))
decidingraces["date"] = pd.to_datetime(decidingraces["date"], format="%Y-%m-%d")
decidingraces["decidingRace"] = True

In [4]:
results = results.merge(decidingraces, on = ["date"], how = "left")

## Titles won by winning the deciding race

In [5]:
results[(results["position"] == 1) & (results["decidingRace"] == True) & (results["name"] == results["champion"])].groupby("name")["raceId"].nunique().nlargest(50)

name
Michael Schumacher    4
Lewis Hamilton        3
Juan Fangio           3
Sebastian Vettel      2
Alberto Ascari        2
Mika Häkkinen         2
Graham Hill           2
Jim Clark             2
Phil Hill             1
Nino Farina           1
Max Verstappen        1
Alain Prost           1
Kimi Räikkönen        1
Alan Jones            1
Jackie Stewart        1
Emerson Fittipaldi    1
Damon Hill            1
Ayrton Senna          1
Jody Scheckter        1
Name: raceId, dtype: int64

## Circuits that have hosted the most title-deciding races

In [6]:
results[results["decidingRace"] == True].groupby("location")["raceId"].nunique().nlargest(10)

location
Monza             12
Suzuka            11
São Paulo          6
Mexico City        5
Abu Dhabi          4
New York State     3
Nürburg            3
Adelaide           2
Austin             2
Bern               2
Name: raceId, dtype: int64

## For how long have the World Champions competed after winning their last title? 

In [7]:
dateOfLastTitle = pd.Series(results[(results["decidingRace"] == True) & (results["name"] == results["champion"])].groupby(["driverId"])["date"].max(), name="dateOfLastTitle")
results = results.merge(dateOfLastTitle, on = ["driverId"], how = "right")

In [8]:
results["afterLastTitle"] = results["dateOfFarewell"] - results["dateOfLastTitle"]

In [9]:
results.groupby("name")["afterLastTitle"].max().nlargest(55)

name
Fernando Alonso      5719 days
Kimi Räikkönen       5166 days
Jacques Villeneuve   3199 days
Sebastian Vettel     3157 days
Michael Schumacher   3010 days
John Surtees         2877 days
Jenson Button        2779 days
Denny Hulme          2541 days
Graham Hill          2380 days
Alan Jones           2219 days
Emerson Fittipaldi   2191 days
Nino Farina          1834 days
Phil Hill            1820 days
Jack Brabham         1512 days
Keke Rosberg         1492 days
Mario Andretti       1476 days
Nelson Piquet        1463 days
Damon Hill           1113 days
Nigel Mansell        1001 days
James Hunt            945 days
Ayrton Senna          924 days
Jim Clark             883 days
Mika Häkkinen         714 days
Alberto Ascari        637 days
Lewis Hamilton        581 days
Jody Scheckter        392 days
Niki Lauda            378 days
Juan Fangio           336 days
Max Verstappen        189 days
Alain Prost            42 days
Jackie Stewart         28 days
Mike Hawthorn           0 days
Nic

[Jochen Rindt died](https://en.wikipedia.org/wiki/Jochen_Rindt) before winning his first (and last) WC title.

## And for how long have been they waiting for their first title?

In [10]:
dateOfFirstTitle = pd.Series(results[(results["decidingRace"] == True) & (results["name"] == results["champion"])].groupby(["driverId"])["date"].min(), name="dateOfFirstTitle")
results = results.merge(dateOfFirstTitle, on = ["driverId"], how = "right")

In [11]:
results["beforeFirstTitle"] = results["dateOfFirstTitle"] - results["dateOfDebut"]

In [12]:
results.groupby("name")["beforeFirstTitle"].min().nlargest(10)

name
Nigel Mansell    4382 days
Nico Rosberg     3913 days
Mario Andretti   3626 days
Jenson Button    3507 days
Mika Häkkinen    2793 days
Jody Scheckter   2527 days
Max Verstappen   2464 days
Kimi Räikkönen   2422 days
Mike Hawthorn    2310 days
Alain Prost      2093 days
Name: beforeFirstTitle, dtype: timedelta64[ns]

## The most experienced champions

In [13]:
results["beforeLastTitle"] = results["dateOfLastTitle"] - results["dateOfDebut"]

In [14]:
results.groupby("name")["beforeLastTitle"].min().nlargest(10)

name
Alain Prost          5005 days
Lewis Hamilton       4991 days
Niki Lauda           4816 days
Michael Schumacher   4753 days
Nigel Mansell        4382 days
Jack Brabham         4068 days
Nico Rosberg         3913 days
Graham Hill          3822 days
Mario Andretti       3626 days
Jenson Button        3507 days
Name: beforeLastTitle, dtype: timedelta64[ns]

## Seasons and GPs with most and least champions competing

In [15]:
champions = results["champion"].drop_duplicates().to_list()

In [16]:
champResults = results[results.name.isin(champions)]

In [17]:
champResults.groupby("year")["driverId"].nunique().nlargest(5)

year
1972    8
1978    8
1979    8
1980    8
1966    7
Name: driverId, dtype: int64

In [18]:
champResults.groupby("year")["driverId"].nunique().nsmallest(5)

year
1950    3
1951    3
1952    3
1956    3
1957    3
Name: driverId, dtype: int64

In [19]:
champResults.groupby(["year", "gp"])["driverId"].nunique().nlargest(15)

year  gp                      
1978  Austrian Grand Prix         8
      Canadian Grand Prix         8
      Dutch Grand Prix            8
      German Grand Prix           8
      Italian Grand Prix          8
1980  Austrian Grand Prix         8
      Dutch Grand Prix            8
      Italian Grand Prix          8
1966  Italian Grand Prix          7
1970  British Grand Prix          7
      German Grand Prix           7
1972  Italian Grand Prix          7
      United States Grand Prix    7
1973  British Grand Prix          7
      Canadian Grand Prix         7
Name: driverId, dtype: int64

In [20]:
champResults[champResults["year"] > 1990].groupby(["year", "gp"])["driverId"].nunique().nsmallest(5)

year  gp                   
1994  Hungarian Grand Prix     2
      Italian Grand Prix       2
      Portuguese Grand Prix    2
1995  Pacific Grand Prix       2
1994  Belgian Grand Prix       3
Name: driverId, dtype: int64

## What was the last time two future World Champions debuted in the same race?

In [21]:
champResults.groupby("name")["dateOfDebut"].min().nsmallest(50)

name
Nino Farina          1950-05-13
Juan Fangio          1950-05-13
Alberto Ascari       1950-05-21
Mike Hawthorn        1952-06-22
Jack Brabham         1955-07-16
Graham Hill          1958-05-18
Phil Hill            1958-07-06
John Surtees         1960-05-29
Jim Clark            1960-06-06
Jackie Stewart       1965-01-01
Denny Hulme          1965-05-30
Mario Andretti       1968-10-06
Emerson Fittipaldi   1970-07-18
Niki Lauda           1971-08-15
Jody Scheckter       1972-10-08
James Hunt           1973-06-03
Alan Jones           1975-04-27
Keke Rosberg         1978-03-04
Nelson Piquet        1978-07-30
Alain Prost          1980-01-13
Nigel Mansell        1980-08-17
Ayrton Senna         1984-03-25
Mika Häkkinen        1991-03-10
Michael Schumacher   1991-08-25
Damon Hill           1992-05-03
Jacques Villeneuve   1996-03-10
Jenson Button        2000-03-12
Fernando Alonso      2001-03-04
Kimi Räikkönen       2001-03-04
Nico Rosberg         2006-03-12
Lewis Hamilton       2007-03-18
Seb

In [22]:
champResults[~champResults.name.isin(firstGPdrivers)].groupby("dateOfDebut")["name"].nunique().nlargest(3)

dateOfDebut
2001-03-04    2
1950-05-21    1
1952-06-22    1
Name: name, dtype: int64

In [23]:
champResults[~champResults.name.isin(lastGPdrivers)].groupby("dateOfFarewell")["name"].nunique().nlargest(3)

dateOfFarewell
1980-10-05    2
1986-10-26    2
1955-05-22    1
Name: name, dtype: int64

## The most unsuccessful World Champions, in one way or another

In [25]:
results[results.name.isin(champions) & results.status.isin(dnqs)].groupby("name")["raceId"].nunique().nlargest(5)

name
Keke Rosberg          14
Damon Hill             6
Emerson Fittipaldi     3
Graham Hill            2
Mika Häkkinen          2
Name: raceId, dtype: int64

Keke Rosberg has failed to qualify for 14 GPs!

In [26]:
results[results.name.isin(champions) & results.status.isin(crashes)].groupby("name")["raceId"].nunique().nlargest(5)

name
Michael Schumacher    30
Nelson Piquet         28
Nigel Mansell         28
Kimi Räikkönen        24
Jenson Button         23
Name: raceId, dtype: int64

In [28]:
championsCrashing = results[results.name.isin(champions) & results.status.isin(crashes)].groupby("name")["raceId"].nunique() / results[results.name.isin(champions)].groupby("name")["raceId"].nunique()

In [35]:
championsCrashing.nlargest(10)

name
James Hunt            0.215054
Damon Hill            0.147541
Nigel Mansell         0.145833
Ayrton Senna          0.135802
Nelson Piquet         0.135266
Alan Jones            0.128205
Mario Andretti        0.124031
Jody Scheckter        0.123894
Niki Lauda            0.109195
Jacques Villeneuve    0.109091
Name: raceId, dtype: float64

In [34]:
championsCrashing.nsmallest(10)

name
Juan Fangio           0.019608
Mike Hawthorn         0.021739
Jim Clark             0.027778
Emerson Fittipaldi    0.033557
Jackie Stewart        0.040000
Lewis Hamilton        0.043771
Denny Hulme           0.044643
John Surtees          0.044643
Fernando Alonso       0.049275
Phil Hill             0.057692
Name: raceId, dtype: float64

James Hunt has had the highest entry-to-crash ratio: 21,5%!